In [1]:
from ipywidgets import interact

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
coso_data = pd.read_csv('./data/cleaned_types.csv')

In [3]:
def norm(s, norm_type):
    if norm_type == 'max':
        if np.nanmax(s) == 0:
            return s
        else:
            return s / np.nanmax(s)
    if norm_type == 'std':
        if np.std(s) == 0:
            return s
        else:
            return s / np.std(s)

In [4]:
names = []
for name in list(coso_data['Well ID'].unique()):
    try:
        name / 1
    except TypeError:
        # These wells are missing depths, so we will skip them.
        if name in ['38C9', '47A-8', '84-30']:
            continue
        names.append(name)

In [5]:
@interact(well_name=names, norm_type=['max', 'std'])
def make_heatplot(well_name, norm_type):
    
    # Get a list of the AMUs available
    amu_list = coso_data.columns[7:-5]
    
    # Get the AMUs for a specific well
    X = coso_data.loc[coso_data['Well ID'] == well_name][amu_list]
    X = np.nan_to_num(X.astype(float).values)
    
    # Normalise each AMU (column)
    X2 = np.apply_along_axis(norm, axis=0, arr=X, norm_type=norm_type)
    
    # Get depths for the plots
    depths = coso_data.loc[coso_data['Well ID'] == well_name]['Depth (ft)']
    extents = (0, 6500, np.nanmax(depths), np.nanmin(depths))
    
    # Change the vmax based on the norm used
    if norm_type == 'std':
        ma = 5
    else:
        ma = 1
    
    # Plotting stuff
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(X2, vmax=ma, extent=extents)
    plt.colorbar(im, shrink=0.75)
    
    ax.set_xticks(np.linspace(0, 6500, 19))
    ax.set_xticklabels(np.arange(0, 181, 10))
    
    return None

interactive(children=(Dropdown(description='well_name', options=('15A-17', '23A-17', '23A-19', '24A-8', '337',…